In [25]:
!pip install sentencepiece
!pip install --upgrade transformers


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 2.7 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.34.1
    Uninstalling transformers-4.34.1:
      Successfully uninstalled transformers-4.34.1


In [20]:
import pandas as pd
import numpy as np
import sentencepiece

import json

In [9]:
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import accuracy_score,classification_report,f1_score,precision_score,recall_score
import torch
#from torch.utils.data import Dataset,DataLoader
from seqeval.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from transformers import DebertaTokenizer,RobertaTokenizer,AutoTokenizer
from transformers import AutoModelForTokenClassification,get_cosine_schedule_with_warmup
from transformers import AdamW,TrainingArguments,Trainer,DataCollatorForTokenClassification
from datasets import load_dataset, ClassLabel, Sequence, load_metric
from accelerate import Accelerator

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
TRAIN = 1 
TRAIN_SPLIT = 0.90
RANDOM_SEED = 42
BATCH_SIZES = 1
EPOCHS = 10

In [6]:
datasets = load_dataset('json', data_files='cuad-v1-annotated.json', field='data')
datasets = datasets['train'].train_test_split(test_size=1-TRAIN_SPLIT, seed=RANDOM_SEED)
datasets

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['split_tokens', 'ner_tags', 'id'],
        num_rows: 282
    })
    test: Dataset({
        features: ['split_tokens', 'ner_tags', 'id'],
        num_rows: 32
    })
})

In [7]:
with open('feature_class_labels.json','r') as f:
    label_list=json.load(f)
label_list

['B-AGMT_DATE',
 'B-DOC_NAME',
 'B-PARTY',
 'I-AGMT_DATE',
 'I-DOC_NAME',
 'I-PARTY',
 'O']

In [28]:
from transformers.models.deberta.tokenization_deberta_fast import Deberta-V2-TokenizerFast
tokenizer=DebertaV2TokenizerFast.from_pretrained("microsoft/deberta-v2-xlarge",add_prefix_space=True)

ImportError: cannot import name 'DebertaV2TokenizerFast' from 'transformers.models.deberta.tokenization_deberta_fast' (/home/pg0963/.local/lib/python3.10/site-packages/transformers/models/deberta/tokenization_deberta_fast.py)

In [ ]:
#X['splited_token'].apply(lambda x:[tokenizer(t) for t in x])

In [ ]:

def tokenize_and_align_labels(examples, label_all_tokens=False):
    tokenized_inputs = tokenizer(examples["split_tokens"],
                                 truncation=True,
                                 is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
dataset_tokenized=datasets.map(tokenize_and_align_labels, batched=True, load_from_cache_file=True)

In [ ]:
model=AutoModelForTokenClassification.from_pretrained("microsoft/deberta-v2-xlarge",num_labels=7)

In [ ]:
model.to(device)
#optimizer
optimizer=AdamW(model.parameters(),
                lr=0.0008)

In [ ]:
accelerator=accelerator = ()

In [ ]:
num_train_samples = len(datasets["train"])
warmup_ratio = 0.2 # Percentage of total steps to go from zero to max learning rate
num_cycles=0.8 # The cosine exponential rate

num_training_steps = num_train_samples*EPOCHS/BATCH_SIZES
num_warmup_steps = num_training_steps*warmup_ratio

lr_sched = get_cosine_schedule_with_warmup(optimizer=optimizer,
                                           num_warmup_steps=num_warmup_steps,
                                           num_training_steps = num_training_steps,
                                           num_cycles=num_cycles)

In [ ]:
args = TrainingArguments(output_dir= 'model_train/',
                         evaluation_strategy = "epoch",
                         learning_rate=0.00008,
                         per_device_train_batch_size=BATCH_SIZES,
                         per_device_eval_batch_size=BATCH_SIZES,
                         num_train_epochs=EPOCHS,
                         weight_decay=0.04,
                         lr_scheduler_type = 'cosine',
                         warmup_ratio=0.2,
                         logging_strategy="epoch",
                         save_strategy="epoch",
                         seed=RANDOM_SEED,
                         metric_for_best_model="f1",
                         load_best_model_at_end = True)   # name of the W&B run (optional)

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)]

    # Define the metric parameters
    overall_precision = precision_score(true_labels, true_predictions, zero_division=1)
    overall_recall = recall_score(true_labels, true_predictions, zero_division=1)
    overall_f1 = f1_score(true_labels, true_predictions, zero_division=1)
    overall_accuracy = accuracy_score(true_labels, true_predictions)

    # Return a dictionary with the calculated metrics
    return {
        "precision": overall_precision,
        "recall": overall_recall,
        "f1": overall_f1,
        "accuracy": overall_accuracy,}

In [ ]:
trainer = Trainer(
                model=model,
                args=args,
                train_dataset=dataset_tokenized["train"],
                eval_dataset=dataset_tokenized["test"],
                data_collator=data_collator,
                tokenizer=tokenizer,
                compute_metrics=compute_metrics,
                optimizers=(optimizer, lr_sched)
                )


In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model('model_saved2')

In [ ]:
test_model=AutoModelForTokenClassification.from_pretrained('model_saved2')

In [ ]:
test_model

In [ ]:
pred_trainer=Trainer(model=test_model,
                     args=args,
                     data_collator=data_collator,
                     compute_metrics=compute_metrics,
                     tokenizer=tokenizer
                    )

In [ ]:
# Extract the predictions and produce a classification report
predictions, labels, _ = pred_trainer.predict(dataset_tokenized["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
    ]

# Generate the metrics and display
results = classification_report(true_labels, true_predictions, zero_division=1)
print(results)

In [ ]:
with open(model_saved) as f:
    !touch f